In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [13]:
class MyOptimizer(optim.Optimizer):

    def __init__(self, params, lr):
        self.lr = lr
        super(MyOptimizer, self).__init__(params, {})

    def step(self, scorer, closure=False):
        for param_group in self.param_groups:
            params = param_group['params']
            # 从param_group中拿出参数
            for param in params:
                # 循环更新每一个参数的值
                param.data = param.data - self.lr * scorer * param.grad

In [14]:
class BPR(nn.Module):
    def __init__(self):
        super(BPR, self).__init__()
        
        self.W = None             # user matrix
        self.H = None             # item matrix
        
        self.Wsc = None        # scorer
        self.Hsc = None 
        
        self.rating_exp = {}
        self.rating_exp_mul_H = {}
        
        self.user_items = {}
        self.dev_user_items = {}
        
        self.uid = None
        self.iid = None
        
        self.uid_dict = None      # serialize uid and iid
        self.iid_dict = None      #  {(original id in dataset): (serial_idx)}
        self.uid_dict_rev = None  # reverse key and value
        self.iid_dict_rev = None  #  {(serial_idx): (original id in dataset)}
        
    def _split(self, df, ratio):
        train = pd.DataFrame(columns = df.columns, dtype=int)
        test = pd.DataFrame(columns = df.columns, dtype=int)
        for i in self.uid:
            train_1, test_1 = train_test_split(df[df.iloc[:, 0] == i], train_size = ratio, shuffle = True, random_state = 5)
            train = pd.concat([train, train_1])
            test = pd.concat([test, test_1])
        return train, test
    
    def split(self, df, train_size=0.8, test_size=0.1):
        self.uid = np.asarray(list(set(df.iloc[:,0].values)))
        self.iid = np.asarray(list(set(df.iloc[:,1].values)))
        self.uid.sort()
        self.iid.sort()
        self.uid_dict = dict(zip(self.uid, [i for i in range(len(self.uid))]))
        self.iid_dict = dict(zip(self.iid, [i for i in range(len(self.iid))]))
        self.uid_dict_rev = {v: k for k, v in self.uid_dict.items()}
        self.iid_dict_rev = {v: k for k, v in self.iid_dict.items()}
        
        train, test = self._split(df, train_size)
        test, dev = self._split(test, test_size / (1 - train_size))
        
        return train, test, dev
    
    def generate_train_batch(self, batch, sets):
        train = []
        for b in range(batch):
            u = self.uid[np.random.randint(0, len(self.uid))]
            i = sets[u][np.random.randint(0, len(sets[u]))]
            j = self.iid[np.random.randint(0, len(self.iid))]
            while j in sets[u]:
                j = self.iid[np.random.randint(0, len(self.iid))]
            train.append([self.uid_dict[u], self.iid_dict[i], self.iid_dict[j]])
        return np.asarray(train) 
    
    def scorer_prob(self, uid, iid):              # Softmax probability, uids, iids serial
        if uid not in self.rating_exp.keys():
            r = 0
            h = 0
            for i in self.user_items[self.uid_dict_rev[uid]]:
                temp = torch.exp(torch.dot(self.Wsc[uid], self.Hsc[self.iid_dict[i]]))
                r += temp
                h += temp * self.Hsc[self.iid_dict[i]]
            self.rating_exp[uid] = r
            self.rating_exp_mul_H[uid] = h
        return torch.exp(torch.dot(self.Wsc[uid], self.Hsc[iid])) / self.rating_exp[uid]

    def fit_dds(self, df, dev, k, stepsize=0.05, regulation_rate=0.0001, max_iter=10, batch=10000):
        self.W = nn.Parameter(torch.rand(len(self.uid), k) * 0.01)    # 初始化 W，H
        self.H = nn.Parameter(torch.rand(len(self.iid), k) * 0.01)  
        
        self.Wsc = torch.rand(len(self.uid), k) * 0.01  # 初始化 scorer
        self.Hsc = torch.rand(len(self.iid), k) * 0.01

        for u in self.uid:                                # 创建字典：用户u对应他访问过的所有items集合
            self.user_items[u] = df[df.iloc[:, 0]==u].iloc[:, 1].values
            self.dev_user_items[u] = dev[dev.iloc[:, 0]==u].iloc[:, 1].values     
        
        optimizer = MyOptimizer([self.W, self.H], lr=1)

        for x in range(max_iter):
            
            # Model update
            loss_sum = 0
            W_grad_sum = torch.zeros(len(self.uid), k)
            H_grad_sum = torch.zeros(len(self.iid), k)
            
            uij = self.generate_train_batch(batch, self.user_items)
            
            for u, i, j in uij:
                optimizer.zero_grad()
                loss = -torch.log(torch.sigmoid(torch.sum(self.W[u] * (self.H[i] - self.H[j]))))
                loss.backward()
                
                W_grad_sum += self.W.grad
                H_grad_sum += self.H.grad
            
                optimizer.step(scorer = self.scorer_prob(u, i))
                loss_sum += loss

            # DDS update
            W_grad_dev_sum = torch.zeros(len(self.uid), k)
            H_grad_dev_sum = torch.zeros(len(self.iid), k)

            for u, i, j in self.generate_train_batch(5000, self.dev_user_items):
                optimizer.zero_grad()
                dev_loss = -torch.log(torch.sigmoid(torch.sum(self.W[u] * (self.H[i] - self.H[j]))))
                dev_loss.backward()
            
                W_grad_dev_sum += self.W.grad
                H_grad_dev_sum += self.H.grad
                
            r_W = torch.sum(W_grad_sum * W_grad_dev_sum, dim=1)
            r_H = torch.sum(H_grad_sum * H_grad_dev_sum, dim=1)
            r_W = r_W.unsqueeze(1).repeat(1, k)
            r_H = r_H.unsqueeze(1).repeat(1, k)

            log_prob_W_grad_sum = torch.zeros(len(self.uid), k)
            log_prob_H_grad_sum = torch.zeros(len(self.iid), k)

            for u, i, j in uij:               
                log_prob_W_grad_sum[u] = self.Hsc[i] - self.rating_exp_mul_H[u] / self.rating_exp[u]
                log_prob_H_grad_sum[i] = self.Wsc[u] * (1 - self.scorer_prob(u, i))

            self.Wsc += r_W * log_prob_W_grad_sum
            self.Hsc += r_H * log_prob_H_grad_sum

            self.rating_exp = {}
            self.rating_exp_mul_H = {}

#             if x == max_iter - 1:
            print(f"Iteration: {x+1}, BPR loss: {loss_sum.item() / batch}")
                
    def _predict(self, uid, items, n):
        top_N = []

        for i in range(len(items)):
            user = self.uid_dict[uid]
            item = self.iid_dict[items[i]]
            top_N.append((items[i], torch.dot(self.W[user], self.H[item])))

        return sorted(top_N, key=lambda s: s[1], reverse=True)[:n]

    def NDCG(self, uid, test, n):
        test_user = test[test[:, 0] == uid]
        rating = self._predict(uid, test_user[:, 1], n)
        irating = sorted(test_user[:, 2], reverse=True)
        dcg = 0
        idcg = 0
        if n > len(irating):
            n = len(irating)
        for i in range(n):
            r = test_user[test_user[:, 1] == rating[i][0], 2][0]
            dcg += 1.0 * (2 ** r - 1) / math.log(i + 2, 2)
            idcg += 1.0 * (2 ** irating[i] - 1) / math.log(i + 2, 2)
        return dcg / idcg

    def performance(self, test, n):
        hit = 0
        n_recall = 0
        n_precision = 0
        ndcg = 0
        for i in self.uid:
            unknown_items = np.setdiff1d(self.iid, self.user_items[i])
            known_items = test[test[:, 0] == i][:, 1]

            ru = self._predict(i, unknown_items, n)
            for item, pui in ru:
                if item in known_items:
                    hit += 1
            n_recall += len(known_items)
            n_precision += n
            ndcg += self.NDCG(i, test, n)

        recall = hit / (1.0 * n_recall)
        precision = hit / (1.0 * n_precision)
        ndcg /= len(self.uid)
        return recall, precision, ndcg

In [4]:
df1 = pd.read_csv("./ml-100k/u.data", sep="\t", names=['user id', 'item id', 'rating', 'timestamp'])
df2 = pd.read_csv("./ml-1m/ratings.dat", sep="::", names=['user id', 'item id', 'rating', 'timestamp'], engine='python')

### 100K

In [15]:
model1 = BPR()
train1, test1, dev1 = model1.split(df1)
print(train1.shape)
print(test1.shape)
print(dev1.shape)

(79619, 4)
(9942, 4)
(10439, 4)


In [16]:
%%time
model1.fit_dds(train1, dev1, k = 50)

Iteration: 1, BPR loss: 0.693035498046875
Iteration: 2, BPR loss: 0.692777587890625
Iteration: 3, BPR loss: 0.692345263671875
Iteration: 4, BPR loss: 0.6914646484375
Iteration: 5, BPR loss: 0.689738916015625
Iteration: 6, BPR loss: 0.68580107421875
Iteration: 7, BPR loss: 0.67814130859375
Iteration: 8, BPR loss: nan
Iteration: 9, BPR loss: nan
Iteration: 10, BPR loss: nan
CPU times: total: 2min 17s
Wall time: 3min 9s


In [6]:
n = 10
rec, pre, ndcg = model1.performance(test1, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.13626723223753975
Recall@10: 0.1292496479581573
NDCG@10: 0.8197595966855103


### 1M

In [4]:
model3 = BPR()
model4 = BPR()
train3, test3, dev3 = model3.split(df2)
train4, test4, dev4 = model4.split(df2)
print(train3.shape)
print(test3.shape)
print(dev3.shape)

(797758, 4)
(99692, 4)
(102759, 4)


In [5]:
model3.fit_normal(train3, k = 20)

Iteration: 500, BPR loss: 0.1950905779539366


In [6]:
n = 10
rec, pre, ndcg = model3.performance(test3, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.13279801324503313
Recall@10: 0.08045781005496931
NDCG@10: 0.8144240583321367
